In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [2]:
transform = transforms.ToTensor()

trainset = torchvision.datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
testset = torchvision.datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False)


Failed to download (trying next):
HTTP Error 404: Not Found



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)

        x = x.view(-1, 16*4*4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = LeNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(trainloader):.4f}")


Epoch [1/10], Loss: 0.2874
Epoch [2/10], Loss: 0.0830
Epoch [3/10], Loss: 0.0597
Epoch [4/10], Loss: 0.0456
Epoch [5/10], Loss: 0.0390
Epoch [6/10], Loss: 0.0333
Epoch [7/10], Loss: 0.0289
Epoch [8/10], Loss: 0.0243
Epoch [9/10], Loss: 0.0208
Epoch [10/10], Loss: 0.0170


In [5]:
def test_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

acc = test_accuracy(model, testloader)
print(f"Clean Test Accuracy: {acc:.2f}%")


Clean Test Accuracy: 98.97%


### Adversial Attack

In [7]:
def pgd_attack(model, images, labels,
               epsilon=0.3, alpha=0.01, iters=40,
               random_start=True):

    model.eval()
    images = images.to(device)
    labels = labels.to(device)
    ori_images = images.clone().detach()

    # random start inside epsilon-ball
    if random_start:
        noise = torch.empty_like(images).uniform_(-epsilon, epsilon)
        adv_images = torch.clamp(ori_images + noise, 0, 1).detach()
    else:
        adv_images = ori_images.clone().detach()

    for i in range(iters):
        adv_images.requires_grad = True
        outputs = model(adv_images)
        loss = criterion(outputs, labels)

        model.zero_grad()
        if adv_images.grad is not None:
            adv_images.grad.data.zero_()
        loss.backward()

        # gradient step
        adv_images = adv_images + alpha * adv_images.grad.sign()

        # project into epsilon-ball
        adv_images = torch.max(torch.min(adv_images, ori_images + epsilon), ori_images - epsilon)

        # valid pixel range
        adv_images = torch.clamp(adv_images, 0, 1).detach()

    return adv_images


In [10]:
from tqdm import tqdm

def evaluate_pgd(model, loader, epsilon=0.3, alpha=0.01, iters=40):
    model.eval()
    total = 0
    clean_correct = 0
    adv_correct = 0
    flipped = 0

    for images, labels in tqdm(loader, desc="PGD Evaluation"):
        images, labels = images.to(device), labels.to(device)
        total += labels.size(0)

        # clean predictions
        with torch.no_grad():
            clean_out = model(images)
            _, clean_pred = clean_out.max(1)
            clean_correct += (clean_pred == labels).sum().item()

        # adversarial examples
        adv_images = pgd_attack(model, images, labels,
                                epsilon=epsilon, alpha=alpha, iters=iters)

        with torch.no_grad():
            adv_out = model(adv_images)
            _, adv_pred = adv_out.max(1)
            adv_correct += (adv_pred == labels).sum().item()

        # Attack Success Rate
        flipped += ((clean_pred == labels) & (adv_pred != labels)).sum().item()

    clean_acc = 100 * clean_correct / total
    adv_acc = 100 * adv_correct / total
    asr = flipped / clean_correct

    return clean_acc, adv_acc, asr

In [11]:
clean_acc, adv_acc, asr = evaluate_pgd(model, testloader)
print("Clean Accuracy:", clean_acc)
print("Adversarial Accuracy:", adv_acc)
print("Attack Success Rate (ASR):", asr)

PGD Evaluation: 100%|██████████| 10/10 [01:36<00:00,  9.66s/it]

Clean Accuracy: 98.97
Adversarial Accuracy: 0.01
Attack Success Rate (ASR): 0.99989895928059
